In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from preprocess import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
def extract_hog_features(image):
    # Convert image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute HOG features
    features = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
    return features

In [6]:
dataset_dir = "out250"

# List to store image paths and corresponding labels
image_paths = []
labels = []
hog_features=[]
from tqdm import tqdm
# Iterate through each font type directory
for i,font_type in enumerate(os.listdir(dataset_dir)):
    print("Processing font type ",font_type)
    counter=0
    font_type_dir = os.path.join(dataset_dir, font_type)
    font_label=i
    if os.path.isdir(font_type_dir):
        # Iterate through each image in the font type directory
        for image_file in tqdm (os.listdir(font_type_dir)):
            # counter+=1
            # if counter>1000:
            #     break;
            image_path = os.path.join(font_type_dir, image_file)
            # Read the image
            print("Processing img ",image_file)
            image = cv2.imread(image_path)
            # image = preprocess(image)
            # # print("Preprocessed image shape is",image.shape)
            # image= removeSkew(image)
            # print("Skew-Corrected image shape is ",image.shape)
            if image is not None:
                # Extract HOG features
                currHoG=extract_hog_features(image)
                hog_features.append (currHoG)
                print("HoG shape is ",currHoG.shape)
                # Add HOG features to the list
                image_paths.append(image_path)
                labels.append(font_label)

Processing font type  IBM Plex Sans Arabic


  0%|          | 1/1000 [00:00<01:52,  8.89it/s]

Processing img  0.jpeg
HoG shape is  (133956,)
Processing img  1.jpeg
HoG shape is  (133956,)
Processing img  10.jpeg


  0%|          | 5/1000 [00:00<01:31, 10.86it/s]

HoG shape is  (133956,)
Processing img  100.jpeg
HoG shape is  (133956,)
Processing img  101.jpeg
HoG shape is  (133956,)
Processing img  102.jpeg


...

HoG shape is  (133956,)
Processing img  996.jpeg
HoG shape is  (133956,)
Processing img  997.jpeg


100%|█████████▉| 999/1000 [02:32<00:00,  6.62it/s]

HoG shape is  (133956,)
Processing img  998.jpeg
HoG shape is  (133956,)
Processing img  999.jpeg


100%|██████████| 1000/1000 [02:32<00:00,  6.57it/s]

HoG shape is  (133956,)


In [12]:
# save in pkl 
# import pickle
# with open('hog_features_nocrop.pkl', 'wb') as f:
#     pickle.dump(hog_features, f)

In [13]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)


In [14]:
# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

In [9]:
# Train SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [10]:

# Predict on the test set
y_pred = svm_model.predict(X_test)

In [11]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6675


In [ ]:
import joblib

# Save the trained SVM model to a file
joblib.dump(svm_model, 'svm_model_linear.pkl')


#loaded_model = joblib.load('svm_model.pkl')

['svm_model_linear.pkl']

In [ ]:
#rbf
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

In [ ]:
y_pred = svm_model.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)